In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import tensorflow as tf
import random
import math

In [2]:
class Data_preprocessor():
    def __init__(self,data,filter_user=1,filter_item=5):
        self.data = data
        self.filter_user = filter_user  # 사용자(session)의 최소 길이를 지정합니다. 기본값은 1입니다.
        self.filter_item = filter_item  # 아이템의 최소 평가 수를 지정합니다. 기본값은 5입니다.

    def preprocess(self):
        self.filter_()  # 데이터를 전처리합니다.
        return self.train_test_split()  # 전처리된 데이터를 훈련 및 테스트 세트로 나눕니다.

    def filter_(self):
        """
        사용자(session)의 길이가 짧은 경우와 아이템의 평가 수가 적은 경우를 필터링합니다.

        :param filter_user: 사용자의 최소 session 길이입니다. 기본값은 1입니다.
        :param filter_item: 아이템의 최소 평가 수입니다. 기본값은 5입니다.
        :return: 데이터프레임
        """
        session_lengths = self.data.groupby('user_id').size()
        self.data = self.data[np.in1d(self.data['user_id'], session_lengths[session_lengths > 1].index)]  # 길이가 2 미만인 session을 필터링합니다.
        print("남은 데이터: %d"%(len(self.data)))
        item_supports = self.data.groupby('business_id').size()  # 각 아이템의 평가 수를 집계합니다.
        self.data = self.data[np.in1d(self.data['business_id'], item_supports[item_supports > 5].index)]  # 평가 수가 5 미만인 아이템을 필터링합니다.
        print("남은 데이터: %d"%(len(self.data)))
        """한 번의 클릭만 있는 사용자도 필터링합니다. 아이템을 필터링할 때 단일 클릭 사용자가 발생할 수 있기 때문입니다."""
        session_lengths = self.data.groupby('user_id').size()
        self.data = self.data[np.in1d(self.data['user_id'], session_lengths[session_lengths > 1].index)]
        print("남은 데이터: %d"%(len(self.data)))

    def train_test_split(self,time_range=86400):
        """
        훈련 및 테스트 데이터 세트로 분할합니다.

        :param time_range: session이 이 기간 내에 있으면 테스트 데이터로 분류됩니다. 기본값은 86400(1일)입니다.
        :return: 두 개의 데이터프레임으로 이루어진 튜플
        """
        tmax = self.data['timestamp'].max()
        session_tmax = self.data.groupby('user_id')['timestamp'].max()
        train = self.data[np.in1d(self.data['user_id'] , session_tmax[session_tmax <= tmax - 86400].index)]
        test = self.data[np.in1d(self.data['user_id'] , session_tmax[session_tmax > tmax - 86400].index)]
        print("훈련 데이터 집계:  session 개수:%d , 아이템 개수:%d , 이벤트 수:%d"%(train['user_id'].nunique(),train['business_id'].nunique(),len(train)))
        """
        협업 필터링 특성상, 테스트 데이터에 훈련 데이터에 없는 아이템이 포함되어 있으면 해당 아이템을 필터링합니다.
        """
        test = test[np.in1d(test['business_id'], train['business_id'])]
        tslength = test.groupby('user_id').size()
        test = test[np.in1d(test['user_id'], tslength[tslength >= 2].index)]
        print("테스트 데이터 집계:  session 개수:%d , 아이템 개수:%d , 이벤트 수:%d"%(test['user_id'].nunique(),test['business_id'].nunique(),len(test)))

        return train


In [2]:
class BPR():
    '''
    parameter
    train_sample_size : 훈련 시, 각 양성 샘플당 샘플링할 음성 샘플의 수
    test_sample_size : 테스트 시, 각 양성 샘플당 샘플링할 음성 샘플의 수
    num_k : 아이템 임베딩의 차원 크기
    evaluation_at : recall@n, 즉 양성 샘플이 상위 몇 개여야 올바른 추천으로 간주하는지 지정
    '''
    def __init__(self, data, n_epochs=10, batch_size=32, train_sample_size=10, test_sample_size=50, num_k=100, evaluation_at=10):
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.train_sample_size = train_sample_size
        self.test_sample_size = test_sample_size
        self.num_k = num_k
        self.evaluation_at = evaluation_at

        self.data = data
        self.num_user = len(self.data['user_id'].unique())
        self.num_item = len(self.data['business_id'].unique())
        self.num_event = len(self.data)

        self.all_item = set(self.data['business_id'].unique())
        self.experiment = []

        # 아이디가 항상 연속적이지 않기 때문에, 맵을 생성하여 아이디를 정규화합니다.
        user_id = self.data['user_id'].unique()
        self.user_id_map = {user_id[i] : i for i in range(self.num_user)}
        item_id = self.data['business_id'].unique()
        self.item_id_map = {item_id[i] : i for i in range(self.num_item)}
        training_data = self.data.loc[:,['user_id','business_id']].values
        self.training_data = [[self.user_id_map[training_data[i][0]], self.item_id_map[training_data[i][1]]] for i in range(self.num_event)]

        # 데이터 전처리
        self.split_data()  # 데이터를 훈련 데이터와 테스트 데이터로 분할합니다.
        self.sample_dict = self.negative_sample()  # 각 훈련 데이터 (사용자, 아이템+)에 대해 BPR 훈련을 위해 10개의 음성 아이템을 샘플링합니다.

        # 모델 생성
        self.build_model()  # TensorFlow 그래프를 구축합니다.
        self.sess = tf.Session()  # 세션을 생성합니다.
        self.sess.run(tf.global_variables_initializer())

    def split_data(self):
        user_session = self.data.groupby('user_id')['business_id'].apply(set).reset_index().loc[:,['business_id']].values.reshape(-1)
        self.testing_data = []
        for index, session in enumerate(user_session):
            random_pick = self.item_id_map[random.sample(session, 1)[0]]
            self.training_data.remove([index, random_pick])
            self.testing_data.append([index, random_pick])

    def negative_sample(self):
        user_session = self.data.groupby('user_id')['business_id'].apply(set).reset_index().loc[:,['business_id']].values.reshape(-1)
        sample_dict = {}

        for td in self.training_data:
            sample_dict[tuple(td)] = [self.item_id_map[s] for s in random.sample(self.all_item.difference(user_session[td[0]]), self.train_sample_size)]

        return sample_dict

    def build_model(self):
        self.X_user = tf.placeholder(tf.int32, shape=(None, 1))
        self.X_pos_item = tf.placeholder(tf.int32, shape=(None, 1))
        self.X_neg_item = tf.placeholder(tf.int32, shape=(None, 1))
        self.X_predict = tf.placeholder(tf.int32, shape=(1))

        user_embedding = tf.Variable(tf.truncated_normal(shape=[self.num_user, self.num_k], mean=0.0, stddev=0.5))
        item_embedding = tf.Variable(tf.truncated_normal(shape=[self.num_item, self.num_k], mean=0.0, stddev=0.5))

        embed_user = tf.nn.embedding_lookup(user_embedding, self.X_user)
        embed_pos_item = tf.nn.embedding_lookup(item_embedding, self.X_pos_item)
        embed_neg_item = tf.nn.embedding_lookup(item_embedding, self.X_neg_item)

        pos_score = tf.matmul(embed_user, embed_pos_item, transpose_b=True)
        neg_score = tf.matmul(embed_user, embed_neg_item, transpose_b=True)

        self.loss = tf.reduce_mean(-tf.log(tf.nn.sigmoid(pos_score - neg_score)))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.loss)

        predict_user_embed = tf.nn.embedding_lookup(user_embedding, self.X_predict)
        self.predict = tf.matmul(predict_user_embed, item_embedding, transpose_b=True)

    def fit(self):
        self.experiment = []
        for epoch in range(self.n_epochs):
            np.random.shuffle(self.training_data)
            total_loss = 0
            for i in range(0, len(self.training_data), self.batch_size):
                training_batch = self.training_data[i:i+self.batch_size]
                user_id = []
                pos_item_id = []
                neg_item_id = []
                for single_training in training_batch:
                    for neg_sample in list(self.sample_dict[tuple(single_training)]):
                        user_id.append(single_training[0])
                        pos_item_id.append(single_training[1])
                        neg_item_id.append(neg_sample)

                user_id = np.array(user_id).reshape(-1, 1)
                pos_item_id = np.array(pos_item_id).reshape(-1, 1)
                neg_item_id = np.array(neg_item_id).reshape(-1, 1)

                _, loss = self.sess.run([self.optimizer, self.loss],
                            feed_dict={self.X_user: user_id, self.X_pos_item: pos_item_id, self.X_neg_item: neg_item_id}
                            )
                total_loss += loss

            num_true = 0
            for test in self.testing_data:
                result = self.sess.run(self.predict, feed_dict={self.X_predict: [test[0]]})
                result = result.reshape(-1)
                if (result[[self.item_id_map[s] for s in random.sample(self.all_item, self.test_sample_size)]] > result[test[1]]).sum() + 1 <= self.evaluation_at:
                    num_true += 1

            print("epoch:%d , loss:%.2f , recall:%.2f" % (epoch, total_loss, num_true/len(self.testing_data)))
            self.experiment.append([epoch, total_loss, num_true/len(self.testing_data)])


In [ ]:
if __name__ == "__main__":
    data = pd.read_csv('ratings_small.csv')
    dp = Data_preprocessor(data)
    processed_data = dp.preprocess()

    bpr = BPR(processed_data)
    bpr.fit()